<a href="https://colab.research.google.com/github/voirinprof/python-geo-colab/blob/main/Appliquer_Random_Forest_(GEE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutoriel : Appliquer Random Forest avec Google Earth Engine et Google Colab


### Objectif

1. Configurer notre environnement.
2. Charger une image satellite avec GEE.
3. Entraîner un modèle Random Forest dans GEE.
4. Appliquer la classification à l’image entière.
5. Visualiser les résultats avec geemap et exporter la carte prédite.


Prérequis : Un compte Google (pour Colab, GEE et Google Drive).

## Installer GEE et scikit-learn dans Colab

In [1]:
# Installer et configurer GEE et scikit-learn
!pip install earthengine-api scikit-learn

## Importer les libs utiles

In [2]:
# Importer les bibliothèques
import ee
import geemap

Se connecter à GEE

In [3]:

# Authentification (exécutez cette cellule et suivez les instructions)
ee.Authenticate()  # Ouvre une fenêtre pour se connecter à votre compte Google
ee.Initialize(project="gmq580")  # Initialise GEE avec vos identifiants

# il peut être nécessaire de créer un projet dans Google Cloud Console

print("GEE est prêt à être utilisé !")

GEE est prêt à être utilisé !


## Débuter votre analyse

Définir une zone d’intérêt

In [4]:
# On va utiliser la localisation de Sherbrooke
region = ee.Geometry.Point([-71.888351, 45.404476])  # Centre sur Sherbrooke
area = region.buffer(10000)  # Crée une zone tampon de 10 km autour du point

Charger les données

In [5]:
# Charger une image Sentinel-2 (médiane sur l'été 2023)
sentinel = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterBounds(area) \
            .filterDate('2023-06-01', '2023-08-31') \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .median()

# Sélectionner les bandes spectrales et calculer le NDVI
bands = sentinel.select(['B2', 'B3', 'B4', 'B8'])  # Bleu, Vert, Rouge, NIR
ndvi = sentinel.normalizedDifference(['B8', 'B4']).rename('NDVI')
image = bands.addBands(ndvi)  # Image avec bandes et NDVI

Créer des données d’entraînement

In [9]:
# Simuler des points d'entraînement adaptés à Sherbrooke (0=urbain, 1=végétation, 2=eau)
training_points = ee.FeatureCollection([
    # Urbain : Centre-ville de Sherbrooke
    ee.Feature(ee.Geometry.Point([-71.888351, 45.404476]), {'class': 0}),
    # Urbain : Près de l’Université de Sherbrooke
    ee.Feature(ee.Geometry.Point([-71.9280, 45.3780]), {'class': 0}),
    # Végétation : Parc du Mont-Bellevue
    ee.Feature(ee.Geometry.Point([-71.9167, 45.3890]), {'class': 1}),
    # Végétation : Forêt autour du Lac des Nations
    ee.Feature(ee.Geometry.Point([-71.8750, 45.4150]), {'class': 1}),
    # Eau : Rivière Magog près du centre
    ee.Feature(ee.Geometry.Point([-71.8850, 45.3980]), {'class': 2}),
    # Eau : Lac des Nations
    ee.Feature(ee.Geometry.Point([-71.8700, 45.4100]), {'class': 2})
])

# Échantillonner l'image aux points d'entraînement
training_data = image.sampleRegions(
    collection=training_points,
    properties=['class'],
    scale=10
)

Entraîner le modèle Random Forest

In [10]:
# Définir le classificateur Random Forest dans GEE
classifier = ee.Classifier.smileRandomForest(
    numberOfTrees=100,  # Nombre d'arbres
    seed=42  # Pour la reproductibilité
).train(
    features=training_data,
    classProperty='class',  # Propriété contenant les étiquettes
    inputProperties=['B2', 'B3', 'B4', 'B8', 'NDVI']  # Bandes utilisées
)

# Appliquer le classificateur à l'image entière
classified = image.classify(classifier)

Visualiser les résultats

In [12]:
# Créer une carte interactive avec geemap
Map = geemap.Map(center=[45.404476, -71.888351], zoom=11)

# Ajouter l'image RGB pour référence
rgb_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
Map.addLayer(sentinel, rgb_params, 'Image RGB')

# Ajouter la classification
class_params = {'min': 0, 'max': 2, 'palette': ['gray', 'green', 'blue']}  # Urbain, Végétation, Eau
Map.addLayer(classified, class_params, 'Classification Random Forest')

# Afficher la carte
Map.addLayerControl()
Map

Map(center=[45.404476, -71.888351], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sea…